In [6]:
import os
import pandas as pd
import re

folder = "."
arquivos = [f for f in os.listdir(folder) if f.endswith('.xlsx') and not f.startswith('ANOSFINAIS_curriculo_normalizado')]

linhas = []

def encontrar_coluna(df, padroes):
    """Busca coluna por múltiplos padrões"""
    if isinstance(padroes, str):
        padroes = [padroes]
    
    for col in df.columns:
        nome = re.sub(r"\s+", " ", str(col).upper().strip())
        for padrao in padroes:
            if re.search(padrao, nome):
                return col
    return None

def normalizar_ano(ano_str):
    """Normaliza o formato do ano para 1º Ano, 2º Ano, etc. - Aceita º e °"""
    if not ano_str:
        return ""
    
    ano_clean = re.sub(r"\s+", " ", str(ano_str).strip().upper())
    
    # Aceita tanto º quanto ° (grau) quanto ª
    numero_match = re.search(r"(\d+)[º°ª]?\s*ANO", ano_clean)
    
    if not numero_match:
        # Tenta apenas o número
        numero_match = re.search(r"(\d+)", ano_clean)
    
    if not numero_match:
        return ano_str
    
    numero = numero_match.group(1)
    mapa_anos = {
        "6": "6º Ano",
        "7": "7º Ano", 
        "8": "8º Ano",
        "9": "9º Ano"
    }
    
    return mapa_anos.get(numero, f"{numero}º Ano")

def normalizar_disciplina(disciplina_str):
    """Normaliza o nome da disciplina com acentos corretos"""
    if not disciplina_str:
        return ""
    
    disciplina_clean = re.sub(r"\s+", " ", str(disciplina_str).strip().upper())
    
    mapa_disciplinas = {
        "ARTE": "ARTE",
        "ARTES": "ARTE",
        "MATEMATICA": "MATEMÁTICA",
        "MATEMÁTICA": "MATEMÁTICA",
        "PORTUGUES": "PORTUGUÊS",
        "PORTUGUÊS": "PORTUGUÊS",
        "LINGUA PORTUGUESA": "LÍNGUA PORTUGUESA",
        "LÍNGUA PORTUGUESA": "LÍNGUA PORTUGUESA",
        "CIENCIAS": "CIÊNCIAS",
        "CIÊNCIAS": "CIÊNCIAS",
        "HISTORIA": "HISTÓRIA",
        "HISTÓRIA": "HISTÓRIA",
        "GEOGRAFIA": "GEOGRAFIA",
        "EDUCACAO FISICA": "EDUCAÇÃO FÍSICA",
        "EDUCAÇÃO FISICA": "EDUCAÇÃO FÍSICA",
        "EDUCAÇÃO FÍSICA": "EDUCAÇÃO FÍSICA",
        "ENSINO RELIGIOSO": "ENSINO RELIGIOSO",
        "INGLES": "INGLÊS",
        "INGLÊS": "INGLÊS",
        "LINGUA INGLESA": "LÍNGUA INGLESA",
        "LÍNGUA INGLESA": "LÍNGUA INGLESA"
    }
    
    return mapa_disciplinas.get(disciplina_clean, disciplina_clean)

def extrair_info_titulo(titulo):
    """Extrai ano e disciplina do título - Aceita º, ° e ª"""
    if not titulo:
        return "", ""
    
    titulo = str(titulo).strip()
    
    # Busca por ano - aceita º, °, ª ou sem símbolo
    ano_match = re.search(r"(\d+)[ºª°]?\s*ANO", titulo.upper())
    ano = ano_match.group(0) if ano_match else ""
    
    # Lista de disciplinas conhecidas
    disciplinas_conhecidas = [
        "ARTE", "ARTES", "MATEMÁTICA", "MATEMATICA", "PORTUGUÊS", "PORTUGUES",
        "LÍNGUA PORTUGUESA", "LINGUA PORTUGUESA", "CIÊNCIAS", "CIENCIAS",
        "HISTÓRIA", "HISTORIA", "GEOGRAFIA", "EDUCAÇÃO FÍSICA", "EDUCACAO FISICA",
        "ENSINO RELIGIOSO", "INGLÊS", "INGLES", "LÍNGUA INGLESA", "LINGUA INGLESA"
    ]
    
    # Busca disciplina no título
    disciplina = ""
    titulo_upper = titulo.upper()
    for disc in disciplinas_conhecidas:
        if disc in titulo_upper:
            disciplina = disc
            break
    
    # Se não encontrou, tenta extrair por separadores
    if not disciplina:
        partes = re.split(r"[-–—]", titulo)
        for parte in partes:
            parte_clean = parte.strip().upper()
            if (parte_clean and 
                not re.search(r"\d+[ºª°]?\s*ANO", parte_clean) and
                "BIMESTRE" not in parte_clean and
                len(parte_clean) > 2):
                disciplina = parte_clean
                break
    
    return ano, disciplina

for arquivo in arquivos:
    try:
        caminho = os.path.join(folder, arquivo)
        
        # Lê apenas as primeiras linhas para extrair título
        df_header = pd.read_excel(caminho, header=None, nrows=5)
        
        # Procura o título nas primeiras linhas
        titulo = ""
        for i in range(min(5, len(df_header))):
            for cell in df_header.iloc[i]:
                if isinstance(cell, str) and ("ANO" in str(cell).upper() or "BIMESTRE" in str(cell).upper()):
                    titulo = str(cell)
                    break
            if titulo:
                break
        
        # Extrai informações do título
        ano, disciplina = extrair_info_titulo(titulo)
        
        # Lê a planilha com header na linha 2 (índice 2)
        df = pd.read_excel(caminho, header=2)
        
        # Limpa nomes das colunas
        df.columns = [str(col).strip() if col is not None else f"Coluna_{i}" for i, col in enumerate(df.columns)]
        
        # Preenche células mescladas
        df = df.ffill()
        
        # Remove linhas completamente vazias
        df = df.dropna(how='all')
        
        # Busca colunas
        col_habilidades = encontrar_coluna(df, [
            r"HABILIDADES?",
            r"COMPETENCIAS?",
            r"OBJETIVOS?"
        ])
        
        col_orientacoes = encontrar_coluna(df, [
            r"ORIENTA(Ç|C)(Õ|O)ES.*PEDAG(Ó|O)GICAS?",
            r"ORIENTA(Ç|C)(Õ|O)ES.*DID(Á|A)TICAS?",
            r"SUGEST(Õ|O)ES.*PEDAG(Ó|O)GICAS?",
            r"METODOLOGIA",
            r"SUGEST(Õ|O)ES"
        ])
        
        # Debug
        print(f"\n=== Processando: {arquivo} ===")
        print(f"Título encontrado: '{titulo}'")
        print(f"Ano extraído: '{ano}' -> Normalizado: '{normalizar_ano(ano)}'")
        print(f"Disciplina extraída: '{disciplina}' -> Normalizada: '{normalizar_disciplina(disciplina)}'")
        print(f"Habilidades: {col_habilidades}")
        print(f"Orientações: {col_orientacoes}")
        
        # Processa cada linha
        linhas_adicionadas = 0
        for idx, row in df.iterrows():
            habilidades = ""
            if col_habilidades and col_habilidades in df.columns:
                hab_value = row[col_habilidades]
                if pd.notna(hab_value) and str(hab_value).strip().lower() != 'nan':
                    habilidades = str(hab_value).replace('\n', ' ').replace('\r', ' ').strip()
            
            orientacoes = ""
            if col_orientacoes and col_orientacoes in df.columns:
                orient_value = row[col_orientacoes]
                if pd.notna(orient_value) and str(orient_value).strip().lower() != 'nan':
                    orientacoes = str(orient_value).replace('\n', ' ').replace('\r', ' ').strip()
            
            # Só adiciona se tiver conteúdo relevante
            if habilidades or orientacoes:
                linhas.append({
                    "SEGMENTO": "ANOS FINAIS",
                    "ANO": normalizar_ano(ano),
                    "DISCIPLINA": normalizar_disciplina(disciplina),
                    "HABILIDADES": habilidades,
                    "ORIENTACOES_PEDAGOGICAS": orientacoes
                })
                linhas_adicionadas += 1
        
        print(f"Linhas processadas: {linhas_adicionadas}")
        
    except Exception as e:
        print(f"ERRO ao processar {arquivo}: {str(e)}")
        continue

# Cria DataFrame final e salva
if linhas:
    df_final = pd.DataFrame(linhas)
    
    # Remove linhas completamente vazias
    df_final = df_final[
        (df_final['HABILIDADES'].str.strip() != '') | 
        (df_final['ORIENTACOES_PEDAGOGICAS'].str.strip() != '')
    ]
    
    # Salva arquivo
    nome_arquivo = "ANOSFINAIS_curriculo_normalizado.xlsx"
    df_final.to_excel(nome_arquivo, index=False)
    
    print(f"\n🎉 SUCESSO!")
    print(f"📄 Arquivo criado: {nome_arquivo}")
    print(f"📊 Total de registros: {len(df_final)}")
    
else:
    print("❌ ERRO: Nenhum dado foi processado!")



=== Processando: 6 ano - 1° BIMESTRE - HISTÓRIA.xlsx ===
Título encontrado: '6°ANO - 1°BIMESTRE - HISTÓRIA'
Ano extraído: '6°ANO' -> Normalizado: '6º Ano'
Disciplina extraída: 'HISTÓRIA' -> Normalizada: 'HISTÓRIA'
Habilidades: HABILIDADES
Orientações: ORIENTAÇÕES PEDAGÓGICAS
Linhas processadas: 5

=== Processando: 6 ano - 1º BIMESTRE - EDUCAÇÃO FÍSICA.xlsx ===
Título encontrado: '6º ANO - 1º BIMESTRE -  EDUCAÇÃO FÍSICA'
Ano extraído: '6º ANO' -> Normalizado: '6º Ano'
Disciplina extraída: 'EDUCAÇÃO FÍSICA' -> Normalizada: 'EDUCAÇÃO FÍSICA'
Habilidades: HABILIDADES
Orientações: ORIENTAÇÕES PEDAGÓGICAS
Linhas processadas: 5

=== Processando: 6 ano - 1º BIMESTRE - ENSINO RELIGIOSO.xlsx ===
Título encontrado: '6° ANO - 1° BIMESTRE - ENSINO RELIGIOSO'
Ano extraído: '6° ANO' -> Normalizado: '6º Ano'
Disciplina extraída: 'ENSINO RELIGIOSO' -> Normalizada: 'ENSINO RELIGIOSO'
Habilidades: HABILIDADES
Orientações: ORIENTAÇÕES PEDAGÓGICAS
Linhas processadas: 2

=== Processando: 6-ano-1º-bimestre.